In [1]:
from pyspark.sql.functions import explode, col, lit

# Replace with your actual file path from lakehouse Files explorer
# Right-click file in Files → Copy path → paste here
raw_df = spark.read.json("abfss://massiveproject@onelake.dfs.fabric.microsoft.com/DailyStockLakehouse.Lakehouse/Files/applquicktest.json")

display(raw_df)  # Should show 1 row with "results" array

StatementMeta(, 03552972-58a9-4811-ac64-7546ae4ae46c, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 601c06e5-49a1-4cf3-8105-5b76f31ff305)

In [2]:
from pyspark.sql.functions import explode, col, lit, from_unixtime, to_date

# Flatten the "results" array into rows (one row per daily entry)
flattened_df = raw_df.select(
    col("ticker"),
    explode(col("results")).alias("daily_data")
).select(
    col("ticker"),
    col("daily_data.v").alias("volume"),
    col("daily_data.vw").alias("vwap"),
    col("daily_data.o").alias("open"),
    col("daily_data.c").alias("close"),
    col("daily_data.h").alias("high"),
    col("daily_data.l").alias("low"),
    col("daily_data.t").alias("timestamp_ms"),  # rename for clarity
    col("daily_data.n").alias("transactions")
)

# Correct Unix ms to timestamp: divide by 1000 first
flattened_df = flattened_df.withColumn("timestamp", from_unixtime(col("timestamp_ms") / 1000)) \
                           .drop("timestamp_ms")  # drop original ms column

# Derive Date from correct timestamp
flattened_df = flattened_df.withColumn("Date", to_date(col("timestamp")))

# Cast types for better analysis
flattened_df = flattened_df.withColumn("open", col("open").cast("double")) \
                           .withColumn("high", col("high").cast("double")) \
                           .withColumn("low", col("low").cast("double")) \
                           .withColumn("close", col("close").cast("double")) \
                           .withColumn("volume", col("volume").cast("long")) \
                           .withColumn("transactions", col("transactions").cast("long"))

display(flattened_df)  # Should show ~19 rows, one per daily entry

StatementMeta(, 03552972-58a9-4811-ac64-7546ae4ae46c, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 848bafec-5b04-46c3-9e64-491eeda3fa3c)

In [3]:
flattened_df.write.mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable("DailyStockData_Clean")

StatementMeta(, 03552972-58a9-4811-ac64-7546ae4ae46c, 5, Finished, Available, Finished)

In [4]:
display(spark.table("DailyStockData_Clean").orderBy("Date", ascending=False).limit(5))

StatementMeta(, 03552972-58a9-4811-ac64-7546ae4ae46c, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0fac6d78-0167-4dcb-8dbd-da0fb56086de)